Visualizer for simulated dynamic adaptive networks

Access: datafiles with adj matrices+culture vecs

In [1]:
%matplotlib widget
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import analysis as analysis
from analysis import read_pickle
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [2]:
filename = 'base'
#df = analysis.read_pickle('df_sayama_change_all')
#df = read_pickle('df_group_size10-40_changeall')
df = analysis.read_pickle('df_' + filename)
df_full = analysis.data_cleanup(df)
print(df.shape, df_full.shape)
df_full.columns

(21600, 19) (21600, 19)


Index(['degrees', 'clusterCoeff', 'reciprocity', 'center_1', 'center_2',
       'mean_centers', 'overall_mean_culture', 'giantComponent', 'diam', 'SPL',
       'CD', 'c1_init', 'c2_init', 'c_avg_init', 'std_d', 'std_rs', 'std_rw',
       'tags', 'comps'],
      dtype='object')

## Number of experiments vs Size of Largest Component

In [3]:
x = []
y = []
for i in np.linspace(0, 1.0, 21):
    cum = 1 - i
    con = df_full['giantComponent'] >= cum
    x.append(cum)
    y.append(df_full.loc[con].shape[0])
for i in np.linspace(0,0.15,15):
    cum = 1 - i
    con = df_full['giantComponent'] >= cum
    x.append(cum)
    y.append(df_full.loc[con].shape[0])

In [4]:
plt.close('all')
plt.scatter(x,y)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Start with 0.8 as minimum gc size. Filter df:

In [5]:
df = df_full.loc[df['giantComponent'] >= 0.8]
print(df_full.shape, df.shape)

(21600, 19) (21600, 19)


## Plot base sayama graphs and do linreg

In [6]:
#mplot3d doesnt handle sympy floats
stdd = np.asarray(df['std_d'].values, dtype = "float")
stdrs = np.asarray(df['std_rs'].values, dtype = "float")
stdrw = np.asarray(df['std_rw'].values, dtype = "float")
spl = np.asarray(df['SPL'].values, dtype = "float")
cd = np.asarray(df['CD'].values, dtype = "float")


analysis.plotfig(stdd, cd, spl, 'd', 'base_d')
analysis.plotfig(stdrw, cd, spl, '$r_w$', 'base_rw')
analysis.plotfig(stdrs, cd, spl, '$r_s$', 'base_rs')

In [7]:
df_full['SPL'].min()

1.3726530612244898

In [13]:
#todo plot in right order for overlap
#analysis.plotfig(stdd, cd, spl, 's.d. of $d$', 'base_d')
#analysis.plotfig(stdrs, cd, spl, 's.d. of $r_s$', 'base_rs')
#analysis.plotfig(stdrw, cd, spl, 's.d. of $r_w$', 'base_rw')

## Distance from original cultures plotting

In [8]:
def asnp(series):
    return np.asarray(series.values, dtype = "float")

distdiff = analysis.culture_avg_wrapper(df)
df['distdiff'] = distdiff
surface = df.groupby(['std_d', 'std_rs'])['distdiff'].mean()

In [9]:
surfacedf = pd.DataFrame(surface)

NameError: name 'surface' is not defined

In [ ]:
surfacedf.reset_index(inplace=True)
sdf=surfacedf

plt.close('all')
fig = plt.figure()
fig.set_size_inches(15, 15)
ax = fig.gca(projection='3d')

ax.scatter(stdd, stdrs, distdiff, marker='.')
ax.plot_trisurf(asnp(sdf['std_d']), asnp(sdf['std_rs']), asnp(sdf['distdiff']),
                cmap=cm.coolwarm, linewidth=0, edgecolor='none')
plt.draw()